In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import statistics
from pathlib import Path

import requests
import re
import os

from multiprocess import Pool
import psutil
from tqdm import tqdm

from bs4 import BeautifulSoup

import glob

import matplotlib.pyplot as plt

In [ ]:

path = r"/nas-slot4/schwurbel2"

def set_path(string):
    global path 
    path = string
    

def get_channel_names(path):
    channel_names = []
    for channel in os.listdir(path):
        channel_names.append(channel)

    return channel_names


def run (channel_name):
    import pandas as pd
    import numpy as np
    from pathlib import Path

    import requests
    import re
    import os

    from multiprocess import Pool
    import psutil
    from tqdm import tqdm

    from bs4 import BeautifulSoup

    path = r"/nas-slot4/schwurbel2"
    #dst = r"/Users/luka/Documents/Lehrveranstaltungen/Research Seminar/Data/Dataframes"
    dst = "/data/schwurbelarchiv/extracted_information" 

    def get_channel_names(path):
        channel_names = []
        for channel in os.listdir(path):
            channel_names.append(channel)

        return channel_names

    def find_first_message(content_file):
        lines = str(content_file).split("\n")
        mess_text = "message default clearfix"
        mess_id = "id="
        not_wanted = "message-"
        for i in range(len(lines)):
            line = lines[i]
            if mess_text in line and mess_id in line and not_wanted not in line:
                message_start_id = re.search(r'message(\d+)', line).group(1)
                return int(message_start_id) 

    def find_last_message(content_file):
        lines = str(content_file).split("\n")
        mess_text = "message default clearfix"
        mess_id = "id="
        not_wanted = "message-"
        for i in range(1,len(lines)):
            line = lines[-i]
            if mess_text in line and mess_id in line and not_wanted not in line:
                print(line)
                message_end_id = re.search(r'message(\d+)', line).group(1)
                return int(message_end_id)

    def get_content(path):
        content = []
        # Get the path of each folder
        file_path = os.path.join(path, channel_name)
        # Check if the path is a directory
        if os.path.isdir(file_path):
            # Loop through all the files in the folder
            for file in os.listdir(file_path):
                # Get the path of each file
                file_name = os.path.join(file_path, file)
                #print(file_name)
                # Check if the file name matches the specific one
                if file[:7] == 'message':
                    # Open the specific file
                    with open(file_name, encoding='utf-8') as f:
                        print(file_name)
                        cont = BeautifulSoup(f, "html.parser")
                        content_tup = (cont, find_first_message(cont), find_last_message(cont))
                        content.append(content_tup)
        return content

        
    def get_author(content):
        #create dataframe containing all authors
        df_author = pd.DataFrame(columns=['check', 'group_name', 'posting_date', 'author', 'fwd_author' ])

        pattern = r'title="(.*?)"'
        for n in range(len(content)):

            for i in range(content[n][1], content[n][2]):
                try:
                    if "forwarded body" in str(content[n][0].find('div', id = "message%s"%i)):
                        df_author = pd.concat([df_author, pd.DataFrame([[i, content[n][0].find('div', class_= "text bold").string.strip(), re.search(pattern, str(content[n][0].find('div', id = "message%s"%i).find('div', class_="body"))).group(1), hash(content[n][0].find('div', id = "message%s"%i).find('div', class_="from_name").string.strip()), hash(re.search(r'(?<=from_name">\n).*(?= <span)', str(content[n][0].find('div', id = "message%s"%i).find('div', class_="forwarded body").find('div', class_="from_name")).strip()).group())]], columns=['check', 'group_name', 'posting_date', 'author', 'fwd_author'])])

                    else: 
                        df_author = pd.concat([df_author, pd.DataFrame([[i, content[n][0].find('div', class_= "text bold").string.strip(), re.search(pattern, str(content[n][0].find('div', id = "message%s"%i).find('div', class_="body"))).group(1), hash(content[n][0].find('div', id = "message%s"%i).find('div', class_="from_name").string.strip())]], columns=['check','group_name', 'posting_date', 'author'])])
 
                except:
                    continue
        return df_author



    def get_message(content):
        #create dataframe with all messages
        df_message = pd.DataFrame(columns=['check', 'posting_date', 'message', 'fwd_message', 'fwd_posting_date'])
        pattern = r'title="(.*?)"'


        for n in range(len(content)):

            #pattern to detect text
            text_pattern = re.compile(r'<div class="text">(.*?)</div>', re.DOTALL)

            
            for i in range(content[n][1], content[n][2]):
                try:
                    if "forwarded body" in str(content[n][0].find('div', id = "message%s"%i)):
                        text_content = re.findall(text_pattern, str(content[n][0].find('div', id = "message%s"%i)))[0]
                        df_message = pd.concat([df_message, pd.DataFrame([[i, re.search(pattern, str(content[n][0].find('div', id = "message%s"%i).find('div', class_="body"))).group(1), text_content.replace('<br/>', ' ').replace('\n', ''), re.search(r'<span class="details">(.*?)</span>', str(content[n][0].find('div', id = "message%s"%i).find('div', class_="forwarded body").find('div', class_="from_name")).strip()).group(1)]], columns=['check', 'posting_date', 'fwd_message', 'fwd_posting_date'])])
                    else: 
                        text_content = re.findall(text_pattern, str(content[n][0].find('div', id = "message%s"%i)))[0]
                        df_message = pd.concat([df_message, pd.DataFrame([[i, re.search(pattern, str(content[n][0].find('div', id = "message%s"%i).find('div', class_="body"))).group(1), text_content.replace('<br/>', ' ').replace('\n', '')]], columns=['check', 'posting_date', 'message'])])
                except:
                    continue
        return df_message
        
        

    def get_file(content):
        #create dataframe containing all media files
        df_file = pd.DataFrame(columns=['check', 'posting_date', 'media_file', 'media_file_type', 'fwd_media_file', 'fwd_media_file_type', 'fwd_posting_date'])


        pattern = r'title="(.*?)"'

        for n in range(len(content)):


            for i in range(content[n][1], content[n][2]):
                try:
                    if "forwarded body" in str(content[n][0].find('div', id = "message%s"%i)):
                        if "video" in str(content[n][0].find('div', id = "message%s"%i).find('div', class_="media_wrap clearfix")):
                            df_file = pd.concat([df_file, pd.DataFrame([[i, re.search(pattern, str(content[n][0].find('div', id = "message%s"%i).find('div', class_="body"))).group(1), re.search('href="(.*)"', str(content[n][0].find('div', id = "message%s"%i).find('div', class_="media_wrap clearfix"))).group(1), 'video', re.search(r'<span class="details">(.*?)</span>', str(content[n][0].find('div', id = "message%s"%i).find('div', class_="forwarded body").find('div', class_="from_name")).strip()).group(1)]], columns=['check', 'posting_date', 'fwd_media_file', 'fwd_media_file_type', 'fwd_posting_date'])])

                        elif "photo" in str(content[n][0].find('div', id = "message%s"%i).find('div', class_="media_wrap clearfix")):
                            df_file = pd.concat([df_file, pd.DataFrame([[i, re.search(pattern, str(content[n][0].find('div', id = "message%s"%i).find('div', class_="body"))).group(1), re.search('href="(.*)"', str(content[n][0].find('div', id = "message%s"%i).find('div', class_="media_wrap clearfix"))).group(1), 'photo', re.search(r'<span class="details">(.*?)</span>', str(content[n][0].find('div', id = "message%s"%i).find('div', class_="forwarded body").find('div', class_="from_name")).strip()).group(1)]], columns=['check', 'posting_date', 'fwd_media_file', 'fwd_media_file_type', 'fwd_posting_date'])])

                        elif "Voice message" in str(content[n][0].find('div', id = "message%s"%i).find('div', class_="media_wrap clearfix")):
                            df_file = pd.concat([df_file, pd.DataFrame([[i, re.search(pattern, str(content[n][0].find('div', id = "message%s"%i).find('div', class_="body"))).group(1), re.search('href="(.*)"', str(content[n][0].find('div', id = "message%s"%i).find('div', class_="media_wrap clearfix"))).group(1), 'voice message', re.search(r'<span class="details">(.*?)</span>', str(content[n][0].find('div', id = "message%s"%i).find('div', class_="forwarded body").find('div', class_="from_name")).strip()).group(1)]], columns=['check', 'posting_date', 'fwd_media_file', 'fwd_media_file_type', 'fwd_posting_date'])])

                        else: 

                            continue    
    
                    else:
                        if "video" in str(content[n][0].find('div', id = "message%s"%i).find('div', class_="media_wrap clearfix")):
                            df_file = pd.concat([df_file, pd.DataFrame([[i, re.search(pattern, str(content[n][0].find('div', id = "message%s"%i).find('div', class_="body"))).group(1), re.search('href="(.*)"', str(content[n][0].find('div', id = "message%s"%i).find('div', class_="media_wrap clearfix"))).group(1), 'video']], columns=['check', 'posting_date', 'media_file', 'media_file_type'])])

                        elif "photo" in str(content[n][0].find('div', id = "message%s"%i).find('div', class_="media_wrap clearfix")):
                            df_file = pd.concat([df_file, pd.DataFrame([[i, re.search(pattern, str(content[n][0].find('div', id = "message%s"%i).find('div', class_="body"))).group(1), re.search('href="(.*)"', str(content[n][0].find('div', id = "message%s"%i).find('div', class_="media_wrap clearfix"))).group(1), 'photo']], columns=['check', 'posting_date', 'media_file', 'media_file_type'])])

                        elif "Voice message" in str(content[n][0].find('div', id = "message%s"%i).find('div', class_="media_wrap clearfix")):
                            df_file = pd.concat([df_file, pd.DataFrame([[i, re.search(pattern, str(content[n][0].find('div', id = "message%s"%i).find('div', class_="body"))).group(1), re.search('href="(.*)"', str(content[n][0].find('div', id = "message%s"%i).find('div', class_="media_wrap clearfix"))).group(1), 'voice message']], columns=['check', 'posting_date', 'media_file', 'media_file_type'])])

                        else:
  
                            continue
                except:
                    continue

        return df_file

    def get_dataset(content):
        #joining step by step with different kind of joins to ensure data reliability        
        df_content = pd.merge(get_message(content), get_file(content), on = 'posting_date', how = 'outer')
        df_merge = pd.merge(df_content, get_author(content), on = 'posting_date', how = 'outer')
        df_merge['group_name'], df_merge['author'] = df_merge['group_name'].ffill(), df_merge['author'].ffill()
        df_merge['posting_date'] = pd.to_datetime(df_merge['posting_date'], dayfirst = True, utc=True)
        df_merge["week"] = df_merge["posting_date"].dt.weekofyear
        df_merge["day"] = df_merge["posting_date"].dt.floor('D') + pd.Timedelta(12, unit='h')
        df_merge["weekday"] = df_merge["posting_date"].dt.weekday
        df_merge['message_hash'] = df_merge['message'][df_merge['message'].notna()].apply(hash)
        df_merge['fwd_message_hash'] = df_merge['fwd_message'][df_merge['fwd_message'].notna()].apply(hash)

        return df_merge



        
    channel_list = get_channel_names(r"/nas-slot4/schwurbel2")


    get_dataset(get_content(path)).to_csv(
		Path(dst, channel_name + ".csv.gzip"),
		 index=False,
		  compression="gzip"
	)


   

number_of_cores = 100

# this opens a pool of cores that is then used to compute jobs for you
pool = Pool(number_of_cores)

# get a list of all channels you want to process (fill this in with
# actual working code to get the channel names).
channel_list = get_channel_names(r"/nas-slot4/schwurbel2")

# iterates over the list of channels and runs the function run()
# on one core for each channel. Wrapping the call to pool.imap_unordered()
# in tqdm() gives you a nice progress bar for the process.
'''
for channel in tqdm(
	pool.imap_unordered(func=run, iterable=channel_list),
	total=len(channel_list)
	):
		# note: the "run" function already saves the results to
		# disk, so you don't need to do anything here. Alternatively,
		# you could return the individual channel's df from run() and
		# concatenate all the extracted data here
		pass
        
'''

# it is important to close the pool of processes after completing all jobs
pool.close()


In [ ]:
dst = "/data/schwurbelarchiv/extracted_information"
folder_path = '/data/schwurbelarchiv/extracted_information/*.csv.gzip'

all_files = glob.glob(folder_path)

beginning = "/data/schwurbelarchiv/extracted_information/"
end = ".csv.gzip"


    
folder_dict = {}


base_df = pd.read_csv(all_files[0], compression="gzip", low_memory = False)
file = all_files[0][len(beginning):-len(end)]
        #print(tmp['group_name'].unique())
        #if len(tmp['group_name'].unique()) != 0:
try:
    gname = tmp['group_name'].unique()[0]
    folder_dict[gname]=file
       
except:
    pass

count = 0

for f in all_files[1:]:
    try:
        tmp = pd.read_csv(f, compression="gzip", low_memory = False)
        file = f[len(beginning):-len(end)]
        #print(tmp['group_name'].unique())
        #if len(tmp['group_name'].unique()) != 0:
        gname = tmp['group_name'].unique()[0]
        folder_dict[gname]=file
        base_df = pd.concat([base_df, tmp], axis = 0)
        count += 1
        print(count)
    except:
        count += 1
        continue

base_df["transcribed_message"] = np.nan
base_df = base_df.reset_index(drop=True)
base_df.to_csv("/data/schwurbelarchiv/extracted_information/dataframes/base_df_alt.csv.gzip", compression = 'gzip')

In [ ]:
#base_df = pd.read_csv("/data/schwurbelarchiv/extracted_information/dataframes/base_df.csv.gzip", compression = 'gzip')

In [ ]:
base_df = base_df[base_df['group_name'] != 'LichtkriegerForumChat']

In [ ]:
base_df = base_df[base_df['group_name'] != '🛡 Bismarcks Erben']

In [ ]:
base_df = base_df[base_df['group_name'] != 'Covid-Impftod Corona Impfungen Nebenwirkungen - Wirksamkeit - Notwendigkeit']

In [ ]:
base_df = base_df[base_df['group_name'] != '🤹🏻‍♂️Alle Punkte Verbunden']

In [ ]:
base_df = base_df[base_df['group_name'] != 'KlarTraum-Gruppe Offenlegungen und Aufwachen']

In [ ]:
base_df = base_df[base_df['group_name'] != 'Freiheit ist ein Recht - Kanal !!!']

In [ ]:
base_df['posting_date'] = pd.to_datetime(base_df['posting_date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%d-%m %H:%M:%S') ##one time run

In [ ]:
base_df['posting_date'] = pd.to_datetime(base_df['posting_date'], dayfirst=True, utc = True)

In [ ]:
base_df["week"] = base_df["posting_date"].dt.weekofyear

In [ ]:
base_df['day'] = base_df['posting_date'].dt.floor('D') + pd.Timedelta(12, unit = 'h')

In [ ]:
base_df["weekday"] = base_df["posting_date"].dt.weekday

In [ ]:
def is_group(n_authors):
    if n_authors > 1:
        return "group"
    else:
        return "channel"

In [ ]:
group_info = base_df.groupby(['group_name'])["author"].nunique().reset_index(name="n_authors")

In [ ]:
group_info["is_group"] = group_info["n_authors"].apply(is_group)

In [ ]:
group_info['n_messages_total'] = group_info['group_name'].map(base_df["group_name"].value_counts())

In [ ]:
new_df = base_df.groupby(["group_name"])["message"].count().reset_index(name="n_text_messages")
group_info = pd.merge(group_info, new_df, on='group_name')
new_df = base_df.groupby(["group_name"])["fwd_message"].count().reset_index(name="n_fwd_text_messages")
group_info = pd.merge(group_info, new_df, on='group_name')
new_df = base_df.groupby(["group_name"])["media_file"].count().reset_index(name="n_media")
group_info = pd.merge(group_info, new_df, on='group_name')
new_df = base_df.groupby(["group_name"])["fwd_media_file"].count().reset_index(name="n_fwd_media")
group_info = pd.merge(group_info, new_df, on='group_name')

In [ ]:
new_df = base_df.groupby('group_name')['media_file_type'].apply(lambda x: x[x == "photo"].count()).to_frame()
new_df = new_df.rename(columns={"media_file_type": "photos"})
new_df["videos"] = base_df.groupby('group_name')['media_file_type'].apply(lambda x: x[x == "video"].count()).to_frame()
new_df["voice messages"] = base_df.groupby('group_name')['media_file_type'].apply(lambda x: x[x == "voice message"].count()).to_frame()
new_df["fwd_photos"] = base_df.groupby('group_name')['fwd_media_file_type'].apply(lambda x: x[x == "photo"].count()).to_frame()
new_df["fwd_videos"] = base_df.groupby('group_name')['fwd_media_file_type'].apply(lambda x: x[x == "video"].count()).to_frame()
new_df["fwd_voice messages"] = base_df.groupby('group_name')['fwd_media_file_type'].apply(lambda x: x[x == "voice message"].count()).to_frame()
group_info = pd.merge(group_info, new_df, on='group_name')

In [ ]:
group_info["total_media"] = group_info["n_media"] + group_info["n_fwd_media"]
group_info["total_photos"] = group_info["photos"] + group_info["fwd_photos"]
group_info["total_videos"] = group_info["videos"] + group_info["fwd_videos"]
group_info["total_voice_m"] = group_info["voice messages"] + group_info["fwd_voice messages"]
group_info["media_ratio"] = group_info["total_media"]/group_info["n_messages_total"]

In [ ]:
summary_df = pd.DataFrame({"group_name":["total"] ,
                           "n_authors":[group_info["n_authors"].sum()],
                           "is_group": [str(group_info["is_group"].value_counts()["channel"])
                                        + " channels, " 
                                        + str(group_info["is_group"].value_counts()["group"]) 
                                        + " groups"],
                           "n_messages_total": [group_info["n_messages_total"].sum()], 
                           "n_text_messages": [group_info["n_text_messages"].sum()], 
                           "n_fwd_text_messages": [group_info["n_fwd_text_messages"].sum()],
                           "n_media": [group_info["n_media"].sum()],
                           "n_fwd_media": [group_info["n_fwd_media"].sum()],
                           "photos": [group_info["photos"].sum()],
                           "videos": [group_info["videos"].sum()],
                           "voice messages": [group_info["voice messages"].sum()],
                           "fwd_photos": [group_info["fwd_photos"].sum()],
                           "fwd_videos": [group_info["fwd_videos"].sum()],
                           "fwd_voice messages": [group_info["fwd_voice messages"].sum()],
                           "total_media": [group_info["total_media"].sum()],
                           "total_photos": [group_info["total_photos"].sum()],
                           "total_videos": [group_info["total_videos"].sum()],
                           "total_voice_m": [group_info["total_voice_m"].sum()],
                           "media_ratio": [group_info["total_media"].sum()/group_info["n_messages_total"].sum()] })

In [ ]:
group_info = pd.concat([group_info, summary_df], axis = 0)

In [ ]:
group_info = group_info.reset_index(drop=True)

In [ ]:
#group_info.to_csv("/data/schwurbelarchiv/extracted_information/dataframes/group_info.csv.gzip", compression = 'gzip')